# Romance

In [2]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from .env file
load_dotenv()

# Retrieve the GroQ API key from the environment variable
api_key = os.getenv('GROQ_API_KEY')
model = "llama-3.1-8b-instant"

client = Groq(api_key='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')

# Define the prompting method and language family
language_family = "romance"
method = "self_translation"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user","content": user_prompt,}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def get_groq_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt,}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_groq_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


KeyboardInterrupt: 

# Indo-Aryan

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from .env file
load_dotenv()

# Retrieve the GroQ API key from the environment variable
api_key = os.getenv('GROQ_API_KEY')
model = "llama-3.1-8b-instant"

client = Groq(api_key='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')

# Define the prompting method and language family
language_family = "indo_aryan"
method = "self_translation"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def get_groq_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_groq_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: इजरायली प्रधानमंत्री के बेटे की फोटो जो कि सेना में सेवा दे रहे हैं.
Model Translated Claim: 'A photo of the son of the Israeli prime minister who is currently serving in the military.'
Model Output: There is no definitive way to determine the accuracy of this claim without knowing the current Israeli prime minister and their son's status in the military. However, based on publicly available information, I can provide some context.

In Israel, military service is mandatory for most citizens, including men and women. It's common for children of politicians to serve in the military, just like any other Israeli citizen.

If we consider recent Israeli prime ministers, we have:

*   Benjamin Netanyahu (2015-2021): His son, Yair Netanyahu, is a well-known figure, but there's no clear indication that he's currently serving in the military.
*   Naftali Bennett (2021-2022): There's no publicly available information about his children's military service.
*   Yair Lapid (2022-pres

# Kartvelian

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from .env file
load_dotenv()

# Retrieve the GroQ API key from the environment variable
api_key = os.getenv('GROQ_API_KEY')
model = "llama-3.1-8b-instant"

client = Groq(api_key='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')

# Define the prompting method and language family
language_family = "kartvelian"
method = "self_translation"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def get_groq_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_groq_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: „2 წელია განათლების სამინისტრო დირექტორთა არჩევნებს ატარებს. სკოლების სამეურვეო საბჭოების 80%-ს ერთი...
Model Translated Claim: '“For Two Years Now, the Ministry of Education Has Been Electing of Directors, Governing Councils of 80% of Schools are Chosen in the Same Way, etc.
Model Output: To assess the claim, let's break down the key points mentioned:

1. **"For Two Years Now, the Ministry of Education Has Been Electing of Directors"**
   - This statement suggests a specific practice of electing directors by the Ministry of Education, which has been ongoing for two years. Here, it is crucial to consider if indeed this is a two-year-old practice. If there is any evidence or credible information suggesting that this method of director election has only been in place for less than two years, or if there are other methods of director election that are more prevalent or recognized, this claim could be inaccurate.

2. **"Governing Councils of 80% of Schools are Chosen in the

# Slavic

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from .env file
load_dotenv()

# Retrieve the GroQ API key from the environment variable
api_key = os.getenv('GROQ_API_KEY')
model = "llama-3.1-8b-instant"

client = Groq(api_key='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')

# Define the prompting method and language family
language_family = "slavic"
method = "self_translation"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def get_groq_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_groq_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Original Claim: Парламент Швейцарии одобрил закон о запрете вакцинации граждан
Model Translated Claim: 'The Swiss Parliament approved a law banning the vaccination of citizens.'
Model Output: This prompt seems to imply the ban was law that got through ('approved' implies something had a change made from another initial policy prior) implementation but did not tell when. This was tested in a trial court by compulsory vaccinate families to a very low success. We then went looking on and concluded there could be multiple interpretation based on translation.
 
 There was a national decision regarding public vaccination as part of the BCG vaccination campaign against tuberculosis. There was a compulsory vaccination process for pregnant women at risk of TB.
Inconclusive response


Original Claim: В аптеках продается израильский парацетамол, от которого люди умирают на шестой день приема
Model Translated Claim: None
Error: Translated claim not found.
Original Claim: „Pregovori sa ministarstvo

# Turkic

In [ ]:
import os
import time
import pandas as pd
import json
import re
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from .env file
load_dotenv()

# Retrieve the GroQ API key from the environment variable
api_key = os.getenv('GROQ_API_KEY')
model = "llama-3.1-8b-instant"

client = Groq(api_key='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')

# Define the prompting method and language family
language_family = "turkic"
method = "self_translation"

# Load data
input_file_path = f'../../JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

def get_translation(claim, model=model):
    user_prompt = f"Translate the following claim into English: '{claim}'. You must always make sure your final response is prefixed with 'Translated Claim:' followed by the translated claim."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

def get_groq_response(translated_claim, model=model):
    user_prompt = f"'{translated_claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": user_prompt}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            if retries < 3:
                retries += 1
                wait_time = 4
                print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise e

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Extract final answer from the output text
    final_answer_match = re.search(r'Final Answer:\s*(True|Mostly True|Half True|Mostly False|False)', output, re.IGNORECASE)
    return final_answer_match.group(1).strip().lower() if final_answer_match else None

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    # Translate the claim to English
    translated_claim_raw = get_translation(claim)
    translated_claim_match = re.search(r'Translated Claim:\s*(.*)', translated_claim_raw, re.IGNORECASE)
    translated_claim = translated_claim_match.group(1) if translated_claim_match else None
    print(f"Original Claim: {claim}")
    print(f"Model Translated Claim: {translated_claim}")

    if translated_claim is None:
        print("Error: Translated claim not found.")
        continue
    
    # Evaluate the translated claim
    output_raw = get_groq_response(translated_claim)
    print(f"Model Output: {output_raw}")
    
    # Extract final answer from the cleaned output
    final_answer = clean_output(output_raw)
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response\n\n")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response\n\n")
            summary['correct'] += 1
        else:
            print("Wrong response\n\n")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'translated_claim': translated_claim,
        'output': output_raw,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


KeyboardInterrupt: 